# Quantum mechanics with simple real vectors

Quantum mechanics is usually formulated with a wave function which requires complex numbers, spin vectors which require Pauli matrices, operators which are needed to determine outcome probabilities and a Hamiltonian operator which determines the time evolution. I will examine how much of quantum mechanics can actually be done with _simple real vectors_ from high-school, the usual dot product and a rotation of vectors alone, without using any of the previously mentioned mathematical objects. This reduced complexity will reveal a geometric meaning to quantum mechanics.

Code is be provided to apply and validate the results, which I will extend sometime. While I assume that the overall computational cost does not desrease (and neither increases), this simpler representation may form a more intuitive way to think about interpretations of quantum mechanics.

This notebook will be extended with future topics (e.g. about what makes quantum different from classical, time evolution calculations). Bookmark this notebook and come back if you are interested.

You may email suggestions to [me](mailto:tmtmw275c@relay.firefox.com)

# Wave function as a real vector

To transform a wave function to a real vector we use the density matrix for a pure state. The operation that we commonly will apply to the density matrix is $P=\mathrm{Tr}(\rho A^\dagger)$ for some matrix $A$. It can be identified with an inner product in a vector space as follows:

$$P=\mathrm{Tr}(\rho A^\dagger)= \sum_{ij} \rho_{ij} A^\dagger_{ji} = \sum_{ij} \rho_{ij} A_{ij}^*$$

If we identify the wave function with a column vector $\Psi=(\rho_{11},\rho_{12},\ldots)$ and the other matrix with $A=(A_{11},A_{12},\ldots)$ we already get vectors in a complex vector space. The expression $P$ becomes a dot product $P=\Psi\cdot A$. However, if all our matrices are Hermitian matrices $H_{ij}=H^*_{ji}$, we can continue and create a real vector space, due to the redundancy of the matrix values opposite of the diagonal.

$$
P=\sum_i \rho_{ii}A_{ii}+\sum_{i<j} \left(\rho_{ij}A^*_{ij}+\rho_{ji}A^*_{ji}\right)\\
 =\sum_i \rho_{ii}A_{ii}+\sum_{i<j} \left(\rho_{ij}A^*_{ij}+\rho^*_{ij}A_{ij}\right)\\
 =\sum_i \rho_{ii}A_{ii}+\sum_{i<j} 2 \Re(\rho_{ij}A^*_{ij})\\
 =\sum_i \rho_{ii}A_{ii}+\sum_{i<j} 2 \left(\Re(\rho_{ij})\Re(A_{ij})+\Im(\rho_{ij})\Im(A_{ij})\right)
$$

Therefore we can just stack the diagonal values and the real and imaginary parts of the off-diagonal values into an $n^2$-dimensional real column vector to do the same calculation with a dot product only.

$$\boxed{\Psi=\left(\rho_{11},\rho_{22},\ldots,\rho_{nn},\sqrt{2}\Re(\rho_{12}),\sqrt{2}\Im(\rho_{12}),\sqrt{2}\Re(\rho_{13}),\sqrt{2}\Im(\rho_{13}),\ldots\right)}$$

Any matrix can be converted this way and the expression $P$ becomes a simple dot product

$$P=\Psi \cdot A$$

The probability to measure a certain eigenvector $|\lambda\rangle$ of an observable can be calculated by using $A=|\lambda\rangle\langle\lambda|$ in our expression which is like an obserable with eigenvalue 1 for our desired eigenvector and 0 for all other orthogonal vectors. Therefore converting this the same way into a real vector $\Lambda=|\lambda\rangle\langle\lambda|$ we can write for the probability of measuring a quantum state in another measurement state

$$\boxed{P(\lambda)=\Psi \cdot \Lambda}$$

This equation has a much more natural and geometrically interpretable form than $|\langle \psi|\lambda\rangle|^2$. Later we will see that other needed quantum mechanic operations are simply rotations of these real vectors. Note that it calculates the measurement probabilities, whereas the measurement outcomes (eigenvalues) are treated separately.

# Example for conversion to real vector and measurement probabilities

Let us compare this with conventional quantum mechanics.

In [37]:
from qiskit import QuantumCircuit
import qiskit.quantum_info as qi
from itertools import combinations
from math import sqrt, prod

In [35]:
def qc_to_vec(qc):
    """
    The conversion to a real vector as shown above
    """
    rho=qi.DensityMatrix.from_instruction(qc)
    dim=prod(rho.dims())
    result=[]
    
    for i in range(dim):
        value=rho.data[i,i]
        assert value.imag==0, "Not Hermitian"
        
        result.append(value.real)

    for i, j in combinations(range(dim), r=2):
        value = rho.data[i,j]
        assert rho.data[j, i]==value.conjugate(), "Not Hermitian"
        
        result.append(sqrt(2)*value.real)
        result.append(sqrt(2)*value.imag)
        
    return np.array(result)

In [36]:
qc = QuantumCircuit(2)
qc_to_vec(qc)

(16,) float64 128bytes
array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
qc=QuantumCircuit(2)
qc.draw()

q_0: 
     
q_1:

In [22]:
qi.DensityMatrix.from_instruction(qc)

DensityMatrix([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
               [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]],
              dims=(2, 2))


In [27]:
qc.x(1)

# Quantum postulates

Let us look how the postulates of quantum mechanics look with this new geometric formalism of quantum mechanics. The following postulate's are usually stated for quantum mechanics (R.L.Jaffe 1996, MIT lecture handout)

1. At each instant the state of is system is completely represented by a ket vector $|\psi\rangle$ (in complex vector space)
2. Every observable is described by an operator that acts on the kets
3. The only possible result of a measurement is one of the eigenvalues of the corresponding operator
4. The probability of obtaining a value $\lambda$ with the eigenstate $|\lambda\rangle$ is $P=|\langle \lambda | \psi\rangle|^2$
5. Immediately after the observable has yielded a value $\lambda$, the state of the system is the normalized eigenstate $|\lambda\rangle$
6. The evolution of the system is described by $|\psi(t)\rangle=U(t,t_0)|\psi(t_0)\rangle$ for some unitary operator $U$.

In the alternative formalism introduced here we instead have

1. At each instant the state of is system is completely represented by a vector $\Psi$ (in **real** vector space)
2. Every observable is described by a **vector** that can be **dotted with the state vector**.
3. The only possible result of a measurement is the values that you **independently assign to the observable vectors**.
4. The probability of obtaining a value $\lambda$ with the eigenstate $\Lambda$ is the **dot product** $P=\Psi\cdot\Lambda$
5. Immediately after the observable has yielded a value $\lambda$, the state of the system is the normalized eigenstate $\Lambda$
6. The evolution of the system is described by $\Psi(t)=U\Psi(0)U^\dagger$ for some **rotation** $U$ where the geometric algebra product is used.

We see that overall fewer mathematical concepts are needed in the alternative formalism. The only drawback is, that to conveniently do rotations in arbitrary dimensions we will employ geometric algebra which extends vector spaces with a new type of vector-vector multiplication. (I'll work it out when I have some time)

One interesting point for the interpretation of quantum mechanics is that postulate (4) is like looking up the coefficient of the whole vector in a new basis. A vague idea is that that no collapse is happening at all and we are just looking at a particular basis.

# Derivation of the formalism from principles

Here I show a partial derivation of the the postulates in the alternative formalism from basic principles. With some more principles maybe a theorem like Gleason's theorem can be derived, showing that the real vector formalism is the unique mathematical way to satisfy all principles.

We start with the principles

1. After measuring an outcome $\Lambda$, the probability of measuring this outcome again right after is 1.
2. After measuring an outcome $\Lambda$, the probability of measuring some other "orthogonal" outcomes right after is 0. They constitute disjoint events.
3. Considering all orthogonal, possible outcomes, the summed probabilities must add up to 1 as they are disjoint events.

In equations this would be

$$P(i\rightarrow j)=\delta_{ij}$$
$$\sum_j P(i\rightarrow j)=1$$

This can easily be modeled mathematically with an inner product on vectors:
$$P(i\rightarrow j)=\psi_i \cdot \phi_j$$
and orthonormal vectors
$$\boxed{\psi_i \cdot\psi_j=\delta_{ij}}$$
form the disjoint event outcomes. With more principles, this could be a unique way. Note that $\psi_i$ and $\phi_j$ are not necessarily from the same basis vector set. Different orthonormal bases can be chosen, but in any case we require
$$\boxed{\psi\cdot\psi=1}$$

For the second equation, which due to linearity translates to $\psi_i\cdot\left(\sum_j \phi_j\right)=1$, we require that we **only consider basis vectors which all lie in a particular plane**
$$\boxed{\psi \cdot D=1}$$
where $D$ (diagonal) is some arbitrary vector of length $\sqrt{n}$ (can be arbitrarily chosen as the first step) and all *allowed* orthonormal bases need to satisfy $\sum_i \psi_i=D$. We could always choose $D$ to be along our first basis vector of the vector space.

# What makes quantum mechanics different from classical mechanics?

Work in progress.

The question is why quantum mechanics isn't even more natural than classical mechanics as it satisfies some very basic principles directly. The first guess is that with a large $n$ everything becomes classical and quantum mechanics is old weird, because it is the only few, discrete outcome theory, that satisfies our basic principles.

In [ ]:
Geometric spin construction with plot
Matrix translation; rotation geometric algebra